# How to use fallbacks with create_agent for reliable AI agents

When building production AI agents, API failures and outages are inevitable. The `create_agent` function from LangChain's middleware_agent module supports fallback models to ensure your agents remain operational even when primary LLM providers experience downtime.

This guide demonstrates how to configure robust agents with automatic failover capabilities using the `create_agent` function.

## Why Use Fallbacks with Agents?

LLM API failures can occur due to:
- Rate limiting
- Service outages
- Network issues
- Regional availability problems

By configuring fallback models, your agents can automatically switch to backup providers, ensuring continuous operation.

In [ ]:
%pip install --upgrade --quiet langchain langchain-openai langchain-anthropic langgraph

## Basic Fallback Configuration

The key to using fallbacks with `create_agent` is configuring your model chain **before** passing it to the agent creator.

In [ ]:
from langchain.agents.middleware_agent import create_agent
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.tools import tool

# Define a simple tool
@tool
def get_weather(location: str) -> str:
    """Get the current weather for a location."""
    return f"The weather in {location} is sunny and 75°F"

# Configure models with fallback chain
primary_model = ChatOpenAI(model="gpt-4o", max_retries=0)
fallback_model = ChatAnthropic(model="claude-3-haiku-20240307")

# Create model with fallback support
model_with_fallback = primary_model.with_fallbacks([fallback_model])

# create_agent accepts the fallback-enabled model seamlessly
agent = create_agent(
    model=model_with_fallback,
    tools=[get_weather],
    system_prompt="You are a helpful weather assistant with automatic failover capabilities."
).compile()

print("Agent created successfully with fallback support!")

# Test the agent
result = agent.invoke({
    "messages": [{"role": "user", "content": "What's the weather like in Paris?"}]
})
print(result)

## Best Practices for Agent Fallbacks

### 1. Disable Model-Level Retries

When using fallbacks, disable retries on individual models to ensure faster failover:

```python
# ✅ Good: Fast failover
primary_model = ChatOpenAI(model="gpt-4o", max_retries=0)

# ❌ Avoid: Slow failover due to retries
primary_model = ChatOpenAI(model="gpt-4o", max_retries=3)
```

### 2. Order Models by Performance and Cost

Place your preferred models first:

```python
# ✅ Good: Best model first, then cheaper alternatives
model = ChatOpenAI("gpt-4o").with_fallbacks([
    ChatOpenAI("gpt-4o-mini"),  # Cheaper OpenAI
    ChatAnthropic("claude-3-haiku-20240307")  # Alternative provider
])
```

### 3. Test Fallback Scenarios

Regularly test your fallback configurations to ensure they work when needed.

## Conclusion

The `create_agent` function provides excellent support for fallback models, enabling you to build robust AI agents that can withstand API failures and service interruptions.

**Key Takeaways:**

1. **✅ `create_agent` DOES support fallbacks** - just configure your model chain before passing it
2. **Configure fallbacks at the model level** using the `.with_fallbacks()` method
3. **Disable model-level retries** for faster failover with `max_retries=0`
4. **Order fallbacks strategically** from preferred to backup models
5. **Test your fallback scenarios** to ensure reliability

This approach provides the same reliability benefits as manual fallback implementations while leveraging LangChain's built-in fallback system for cleaner, more maintainable code.

For more information on fallbacks, see the [general fallbacks guide](fallbacks.ipynb).